# Problem Statement

The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

# Ideas

We can't do a full brute force of this, but I've got an idea. We're looking for a 5-set of primes. But since each of these is made up of a pair of primes. So we can solve this with graph theory! We'll create nodes for each number, edges for the prime pairs, and then look for the smallest connected component that has at least 5 primes.



In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import math_utils
from utils import prime_utils

is_prime, primes = prime_utils.load_primes()

In [24]:
prime_pairs = []
num_primes = 1100

for i in range(num_primes - 1):
  a = primes[i]
  for j in range(i + 1, num_primes):
    b = primes[j]
    if is_prime[math_utils.concat(a, b)] and is_prime[math_utils.concat(b, a)]:
      prime_pairs.append((a, b))

print(prime_pairs[:100])

[(3, 7), (3, 11), (3, 17), (3, 31), (3, 37), (3, 59), (3, 67), (3, 73), (3, 109), (3, 137), (3, 191), (3, 229), (3, 271), (3, 331), (3, 359), (3, 373), (3, 449), (3, 467), (3, 499), (3, 541), (3, 557), (3, 607), (3, 613), (3, 617), (3, 673), (3, 701), (3, 719), (3, 733), (3, 739), (3, 823), (3, 929), (3, 947), (3, 1013), (3, 1019), (3, 1033), (3, 1051), (3, 1181), (3, 1193), (3, 1237), (3, 1481), (3, 1531), (3, 1607), (3, 1627), (3, 1657), (3, 1663), (3, 1667), (3, 1699), (3, 1907), (3, 2069), (3, 2143), (3, 2213), (3, 2297), (3, 2377), (3, 2381), (3, 2411), (3, 2441), (3, 2503), (3, 2579), (3, 2707), (3, 2789), (3, 2843), (3, 2917), (3, 2957), (3, 3049), (3, 3119), (3, 3301), (3, 3461), (3, 3469), (3, 3637), (3, 3769), (3, 3911), (3, 3923), (3, 3931), (3, 4019), (3, 4159), (3, 4253), (3, 4583), (3, 4729), (3, 4919), (3, 5051), (3, 5059), (3, 5099), (3, 5171), (3, 5281), (3, 5323), (3, 5381), (3, 5419), (3, 5449), (3, 5507), (3, 5521), (3, 5531), (3, 5573), (3, 5801), (3, 5839), (3, 58

For each of these prime pairs we'll group them together into maximal components. Hopefully, we'll find one with at least 5 primes in it. We'll build our graph using the prime_pairs as node -> node edges. Then we iterate through each node in our graph in the following manner:
1. Build a component with the starting node.
2. Consider each of the adjacent nodes, and see if we can add them to the component.
3. Recurse, considering each of the adjacent nodes adjacent nodes.
4. Once we stop, we've built a "maximal component".

In [25]:
import collections


def can_add_to_component(graph, visited, component, node_to_add):
  if node_to_add in component:
    return False
  if visited[node_to_add]:
    return False
  for node in component:
    if node not in graph[node_to_add]:
      return False
  return True

def build_components(graph, visited, node, components=[], component=[]):
  component.append(node)
  for adj in graph[node]:
    if not can_add_to_component(graph, visited, component, adj):
      continue
    visited[node] = True
    components.append(build_components(graph, visited, adj, components=components, component=component))
  # Deep copy the component
  final_component = component + []
  component.pop()
  return final_component

graph = collections.defaultdict(list)


# Add both edges since this graph is undirected.
for a, b in prime_pairs:
  graph[a].append(b)
  graph[b].append(a)
  
components = {prime: [] for prime in graph.keys()}
  
for prime in list(graph.keys()):
  visited = {prime: False for prime in graph.keys()}
  build_components(graph, visited, prime, components=components[prime])
    
max_len = 0
for prime, component_list in components.items():
  for component in component_list:
    if len(component) > max_len:
      max_len = len(component)
    if len(component) >= 5:
      print(f"component: {component}, sum: {sum(component)}")

print(f"max_len: {max_len}")

  

component: [5197, 13, 5701, 6733, 8389], sum: 26033
component: [5701, 13, 5197, 6733, 8389], sum: 26033
component: [6733, 13, 5197, 5701, 8389], sum: 26033
component: [8389, 13, 5197, 5701, 6733], sum: 26033
max_len: 5
